In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import pickle
import itertools
from numba import jit

In [ ]:
import importlib
import evotsc
import evotsc_lib
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_plot)

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')
gen=1_000_000

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
params = evotsc_lib.read_params(rep_dirs[0])
params['m'] = 2.5 # Temporary fix because the parameter wasn't saved

In [ ]:
@jit(nopython=True)
def run_system_numba_ko(nb_genes: int,
                        init_expr: np.ndarray,
                        inter_matrix: np.ndarray,
                        sigma_basal: float,
                        sigma_opt: float,
                        epsilon: float,
                        m: float,
                        sigma_env: float,
                        id_ko: int) -> np.ndarray:

    step_size = 0.5
    stop_dist = 1e-7
    max_eval_steps = 200

    temporal_expr = np.zeros((max_eval_steps+1, nb_genes))

    # Initial values at t = 0
    temporal_expr[0, :] = init_expr
    temporal_expr[0, id_ko] = 0.0


    # Iterate the system
    it = 1
    cont = True
    while cont:
        prev_expr = temporal_expr[it-1, :]
        sigma_local = inter_matrix @ prev_expr
        sigma_total = sigma_basal + sigma_local + sigma_env

        promoter_activity = 1.0 / (1.0 + np.exp((sigma_total - sigma_opt)/epsilon))

        # We subtract 1 to rescale between exp(-m) and 1
        iter_expr = np.exp(m * (promoter_activity - 1.0))

        nouv_expr = step_size * iter_expr + (1 - step_size) * prev_expr

        # Knockout
        nouv_expr[id_ko] = 0

        temporal_expr[it, :] = nouv_expr

        # Check if we're done
        dist = np.abs(nouv_expr - prev_expr).sum() / nb_genes

        prev_expr = nouv_expr

        if dist < stop_dist:
            cont = False

        if it == max_eval_steps:
            cont = False
        it += 1

    temporal_expr = temporal_expr[:it, :]

    return temporal_expr

In [ ]:
class KoIndividual(evotsc.Individual):
    def __init__(self,
                 orig_indiv: evotsc.Individual,
                 id_ko: int) -> None:
        
        super().__init__([gene.clone() for gene in orig_indiv.genes],
                       orig_indiv.interaction_dist,
                       orig_indiv.interaction_coef,
                       orig_indiv.sigma_basal,
                       orig_indiv.sigma_opt,
                       orig_indiv.epsilon,
                       orig_indiv.m,
                       orig_indiv.selection_coef,
                       orig_indiv.rng)
        
        self.id_ko = id_ko
                    

    def run_system(self, sigma_env):

        init_expr = np.array([gene.basal_expression for gene in self.genes])

        self.inter_matrix = self.compute_inter_matrix()

        return run_system_numba_ko(nb_genes=self.nb_genes,
                                   init_expr=init_expr,
                                   inter_matrix=self.inter_matrix,
                                   sigma_basal=self.sigma_basal,
                                   sigma_opt=self.sigma_opt,
                                   epsilon=self.epsilon,
                                   m=self.m,
                                   sigma_env=sigma_env,
                                   id_ko=self.id_ko)

In [ ]:
def plot_all_kos(rep, rep_dir, gen, sigma):
    
    ko_path = exp_path.joinpath(f'kos_rep{rep:02}')
    ko_path.mkdir(exist_ok=True)
    
    init_indiv = evotsc_lib.get_best_indiv(rep_dir, gen=gen)
    n = 1000  # the number of data points
    _, genome_length = init_indiv.compute_gene_positions(include_coding=True)
    data_positions = np.linspace(0, genome_length, n, dtype=int)

    # Note: we do the difference between supercoiling values of two individuals,
    # so sigma_basal and sigma_env cancel out
    
    init_data = init_indiv.compute_final_sc_at(params['sigma_A'], data_positions)
    evotsc_plot.plot_genome_and_tsc(init_indiv, sigma=sigma, coloring_type='on-off', print_ids=True, 
                                    id_interval=1,
                                    plot_name=ko_path.joinpath(f'ko_genome_and_tsc_gene_0.pdf'))
    
    for i_ko in range(2):# indiv.nb_genes):
        
        ko_indiv = KoIndividual(orig_indiv=init_indiv, id_ko=i_ko)
        ko_data = ko_indiv.compute_final_sc_at(params['sigma_A'], data_positions)

        evotsc_plot.plot_genome_and_tsc(ko_indiv, sigma=params['sigma_A'], coloring_type='on-off',
                                        print_ids=True, ring_data=np.abs(init_data-ko_data), show_bar=True,
                                        bar_text='$\Delta\sigma_{TSC}$', id_interval=1, id_ko=i_ko,
                                        show_plot=False,
                                        plot_name=ko_path.joinpath(f'ko_genome_and_tsc_gene_{i_ko:02}.pdf'))

In [ ]:
for rep, rep_dir in enumerate(rep_dirs[:1]):
    plot_all_kos(rep, rep_dir, gen, sigma=params['sigma_A'])